#📌 Extracción

##**Importación de librerias**

In [1]:
import pandas as pd
import numpy as np
import json
import ast
import warnings

from pandas import json_normalize

warnings.filterwarnings('ignore')

# Cargar archivo local JSON (ya disponible en github)
url_data = 'https://raw.githubusercontent.com/PamelaOrmeno/Challenge_Telecom/main/Data/TelecomX_Data.json'
df = pd.read_json(url_data)

## **Diccionario de datos**

- `customerID`: número de identificación único de cada cliente
- `Churn`: si el cliente dejó o no la empresa
- `gender`: género (masculino y femenino)
- `SeniorCitizen`: información sobre si un cliente tiene o no una edad igual o mayor a 65 años
- `Partner`: si el cliente tiene o no una pareja
- `Dependents`: si el cliente tiene o no dependientes
- `tenure`: meses de contrato del cliente
- `PhoneService`: suscripción al servicio telefónico
- `MultipleLines`: suscripción a más de una línea telefónica
- `InternetService`: suscripción a un proveedor de internet
- `OnlineSecurity`: suscripción adicional de seguridad en línea
- `OnlineBackup`: suscripción adicional de respaldo en línea
- `DeviceProtection`: suscripción adicional de protección del dispositivo
- `TechSupport`: suscripción adicional de soporte técnico, menor tiempo de espera
- `StreamingTV`: suscripción de televisión por cable
- `StreamingMovies`: suscripción de streaming de películas
- `Contract`: tipo de contrato
- `PaperlessBilling`: si el cliente prefiere recibir la factura en línea
- `PaymentMethod`: forma de pago
- `Charges.Monthly`: total de todos los servicios del cliente por mes
- `Charges.Total`: total gastado por el cliente

##**Exploración inicial + validación del equema**

### **Info DataFrame**

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB


###**Dtypes DataFrame**

In [3]:
df.dtypes

,0
customerID,object
Churn,object
customer,object
phone,object
internet,object
account,object


###**Head DataFrame**

In [4]:
df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


#🔧 Transformación

###**Limpieza básica de Strings**

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB


In [6]:
# --- 1. Desanidar columnas si aún no ha sido hecho
if 'customer' in df.columns:
    df_customer = pd.json_normalize(df['customer'])
    df_phone = pd.json_normalize(df['phone'])
    df_internet = pd.json_normalize(df['internet'])
    df_account = pd.json_normalize(df['account'])

    df_flat = pd.concat([
        df[['customerID', 'Churn']],
        df_customer,
        df_phone,
        df_internet,
        df_account
    ], axis=1)

    df = df_flat.copy()
    print("✅ Datos desanidados correctamente.")
else:
    print("🔁 Los datos ya están desanidados. No se vuelve a ejecutar.")

✅ Datos desanidados correctamente.


In [8]:
# --- 2. Normalizar nombres de columnas
df.columns = df.columns.str.strip().str.lower()


In [9]:
# --- 3. Convertir columnas "yes"/"no" a 1 y 0 de forma segura
columnas_binarias = [
    'partner', 'dependents', 'phoneservice', 'multiplelines',
    'onlinesecurity', 'onlinebackup', 'deviceprotection',
    'techsupport', 'streamingtv', 'streamingmovies',
    'paperlessbilling'
]

for col in columnas_binarias:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.lower().replace({'yes': 1, 'no': 0})
    else:
        print(f"⚠️ Columna no encontrada: {col}")

In [11]:
# --- 4. Convertir 'churn' a binario
if 'churn' in df.columns:
    df['churn'] = df['churn'].astype(str).str.strip().str.lower().replace({'yes': 1, 'no': 0})

In [13]:

# --- 5. Convertir columnas numéricas
df['charges.monthly'] = pd.to_numeric(df['charges.monthly'], errors='coerce')
df['charges.total'] = pd.to_numeric(df['charges.total'], errors='coerce')

In [15]:
# --- 6. Crear columna de gasto diario estimado
df['cuentas_diarias'] = df['charges.total'] / 30

In [17]:
# --- 7. Verificar resultados
print("\n📊 Verificación final:")
print(df.info())
print(df.head())



📊 Verificación final:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerid        7267 non-null   object 
 1   churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   seniorcitizen     7267 non-null   int64  
 4   partner           7267 non-null   int64  
 5   dependents        7267 non-null   int64  
 6   tenure            7267 non-null   int64  
 7   phoneservice      7267 non-null   int64  
 8   multiplelines     7267 non-null   object 
 9   internetservice   7267 non-null   object 
 10  onlinesecurity    7267 non-null   object 
 11  onlinebackup      7267 non-null   object 
 12  deviceprotection  7267 non-null   object 
 13  techsupport       7267 non-null   object 
 14  streamingtv       7267 non-null   object 
 15  streamingmovies   7267 non-null   object 
 16  contract          7

#📊 Carga y análisis

#📄Informe final

# Task
Explain the error in the selected code, fix it by flattening the nested columns, and incorporate the changes into the existing code.

## Flatten nested columns

### Subtask:
Extract data from nested dictionary columns into new, top-level columns.


**Reasoning**:
The current error is due to trying to access columns that don't exist in the dataframe. This is because the data is in a nested dictionary format. To fix this, I need to extract the data from these nested columns and create new columns in the dataframe. I will use `json_normalize` to flatten the nested dictionary columns ('customer', 'phone', 'internet', and 'account') and merge the results back into the original dataframe `df` based on 'customerID'.

